# Hello World Detection

A very basic introduction to OpenVINO that shows how to do text detection on a given IR model.

TBD

## Imports

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from openvino.inference_engine import IECore

## Load the network

In [ ]:
ie = IECore()
net = ie.read_network(
    model="model/horizontal-text-detection-0001.xml"
)
exec_net = ie.load_network(net, "CPU")

output_layer_ir = next(iter(exec_net.outputs))
input_layer_ir = next(iter(exec_net.input_info))

## Load an Image

In [ ]:
# Text detection models expects image in BGR format
image = cv2.imread("data/paper.jpg")

# N,C,H,W = batch size, number of channels, height, width
N, C, H, W = net.input_info[input_layer_ir].tensor_desc.dims

# Resize image to meet network expected input sizes
resized_image = cv2.resize(image, (W, H))  

# Reshape to network input shape
input_image = np.expand_dims(
    resized_image.transpose(2, 0, 1), 0
)  

plt.imshow(image)

## Do Inference

In [ ]:
result = exec_net.infer(inputs={input_layer_ir: input_image})

# Extract list of boxes from results
boxes = result['boxes']

# Remove zero only boxes
boxes = boxes[~np.all(boxes==0, axis=1)]

In [ ]:
# For each detection, the description has the format: [x_min, y_min, x_max, y_max, conf]
# Image passed here is in BGR format with changed width and height. To display it in colors expected by matplotlib we use cvtColor funtion

def convert_result_to_image(resized_image, boxes): 
    colors = {'red': (255, 0, 0), 'green': (0, 255, 0)} 
 
    resized_image_rgb = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB) 
    for box in boxes: 
        conf = box[-1]
        if conf > 0.4: 
            (x_min, y_min, x_max, y_max, conf) = map(int, box) 
            resized_image_rgb = cv2.rectangle( 
                resized_image_rgb, 
                (x_min, y_min), 
                (x_max, y_max), 
                colors['green'], 
                3
            ) 
            resized_image_rgb = cv2.putText( 
                resized_image_rgb, 
                f"confidence: {conf:.2f}", 
                (x_min, y_min - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.8, 
                colors['red'], 
                1, 
                cv2.LINE_AA
            ) 
    return cv2.resize(resized_image_rgb, (image.shape[-2:][::-1]))
    

plt.imshow(convert_result_to_image(resized_image, boxes))